In [95]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark=SparkSession.builder.appName('kaggel').getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType


Sale_path=r"C:\Users\Dell\Desktop\sales_project\sales.csv.txt"
Menu_path=r"C:\Users\Dell\Desktop\sales_project\menu.csv.txt"

Sales_schema = StructType([
    StructField("Product_id", IntegerType(), True),        # Integer
    StructField("Customer_id", StringType(), True),  
    StructField("Order_date", DateType(), True),  # Datetime
    StructField("Location", StringType(), True),  
    StructField("Source_order", StringType(), True)
])
Menu_schema=StructType([
    StructField("Product_id",IntegerType(),True),
    StructField("Product_name",StringType(),True),
    StructField("Product_price",StringType(),True)
])


df_sales= spark.read.csv(Sale_path,inferSchema=True,schema=Sales_schema,header="False")
df_menu= spark.read.csv(Menu_path,inferSchema=True,schema=Menu_schema,header="False")

Add Year, Month, Quater Column in Sales data frame


In [61]:
df_sales=df_sales.withColumn("Year",year(df_sales["Order_date"]))
df_sales=df_sales.withColumn("Month",month(df_sales["Order_date"]))
df_sales=df_sales.withColumn("Quater",quarter(df_sales["Order_date"]))
# df_sales.show()

Total Amount Spend By each Customer 

In [74]:
df_amount=df_sales.join(df_menu,"Product_id").groupBy("Product_name").agg(sum("Product_price")).orderBy("Product_name")
# df_amount.show()

Total amount Spend By Food category


In [ ]:
df_amount=df_sales.join(df_menu,"Product_id").groupBy("Month").agg(sum("Product_price")).orderBy("Month")
# df_amount.show()

Total amount Spend in each month

In [ ]:
df_amount=df_sales.join(df_menu,"Product_id").groupBy("Month").agg(sum("Product_price")).orderBy("Month")
# df_amount.show()

Top 3 selling Item

In [133]:
# df_amount=df_sales.join(df_menu,"Product_id").groupBy("Product_name").agg(count("Product_id")).orderBy(count("Product_id").desc()).show()

from pyspark.sql import Window 
from pyspark.sql.functions import *

df_join=df_sales.join(df_menu,"Product_id")
df_count=df_join.groupBy("Product_name").agg(count("Product_id").alias("count"))


window = Window.orderBy(col("count").desc())

df_new=df_count.withColumn("rank",dense_rank().over(window))
# df_new.filter(col("rank")<=3).show()


total sales in each country 

In [146]:
df_revenu=df_sales.join(df_menu,"Product_id").groupBy('location').agg(sum("Product_price").alias("Total_sales"))\
    .orderBy(sum("Product_price").desc())

# df_revenu.show()

total sales by each source

In [ ]:
df_source=df_sales.join(df_menu,"Product_id").groupBy("Source_order").agg(sum("Product_price").alias("Total_sales"))\
    .orderBy(sum("Product_price").desc())
